In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import nglview as nv
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.display import display, display_markdown
from ipywidgets import Layout, HTML
from pathlib import Path

import parmed as pmd
import re

import hublib.use
from hublib.ui import FileUpload, Download
from hublib.cmd import runCommand

from scipy.ndimage import gaussian_filter

%matplotlib notebook
%use gromacs-2018.4
%use pymol-1.8.4

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)


HTMLButtonPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{link}" target="_blank" >
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:100px; background-color:#E9E9E9; font-size:10pt; color:black">{text}</button>
</a>
</body>
</html>
'''

HTMLDeadPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:100px; background-color:#E9E9E9; font-size:10pt; color:#D2D2D2">{text}</button>
</body>
</html>
'''

forbidden_strings = ["..", "/", "\\", " ", "~"]

<IPython.core.display.Javascript object>

In [3]:
class pigment_class:
    def __init__(self, name, BaseNames, OptNames):
        self.name = name
        self.BaseNames = BaseNames
        self.OptNames = OptNames
        
# XNames is the list of atoms names that should be used to identify the pigment type 
# within a given class. 
class pigment_type:
    def __init__(self, ResName, StdName, pClass, XAtNames, NSDNames, TrESPNames, Q10, Q00, Q11, diplength):
        self.pclass = pClass,
        self.name = ResName
        self.stdname = StdName
        self.xnames = XAtNames
        self.nnames = NSDNames
        self.tnames = TrESPNames
        self.q10 = Q10
        self.q00 = Q00
        self.q11 = Q11
        self.diplength = diplength
        
class pigment:
    def __init__(self, idx, ptype, alist, resname, chain, resnum, atnames, atcoords, residue):
        self.idx = idx
        self.ptype = ptype
        self.alist = alist
        self.resname = resname
        self.chain = chain
        self.resnum = resnum
        self.atnames = atnames
        self.atcoords = atcoords
        self.widget = []
        self.res = residue

# Returns a list of all residue indices (in struc)
# that contain all required atoms
def find_pigments(NameList, instruc):

    SelStr = "(:0-"+str(len(instruc.residues)) + ")"
    for ndx in range(1, len(NameList)):
        tstruc = instruc.view[SelStr + ' & (@'+NameList[ndx] + ")"]
        SelNdx = []
        for at in tstruc:
            SelNdx.append(at.residue.idx)
        SelStr = "(:" + ",".join([str(i+1) for i in SelNdx]) + ")"
        if len(SelNdx)==0:
            break

    return SelNdx
    
# Now check which pigment types can be *excluded* based on the atoms
# that are present in each pigment. NB: We do *not* require all atoms
# to be present. 
# The return argument AList is a list of pigment types that *are* allowed. 
def eliminate_types(PigNdcs, PClass, TypeList, UNK, instruc):
    
    AList = []
    for ndx in PigNdcs:
        tstruc = instruc[':'+str(ndx+1)]
        
        # For each atom in the pigment, check whether it's allowed
        # under each pigment type. 
        TypeAllowed = [1 for i in range(0,len(TypeList))]
        for at in tstruc:
            # If not a Hydrogen...we always ignore hydrogens
            if at.name[0]!='H':
                # First check if it's a base atom or optional atom. If not, we check 
                # the characteristic atoms for each pigment. 
                if (PClass.BaseNames.count(at.name)==0) and (PClass.OptNames.count(at.name)==0):
                    for tndx in range(0, len(TypeList)):
                        if TypeList[tndx].xnames.count(at.name)==0:
                            TypeAllowed[tndx] = 0
        
        alist = []
        for tndx in range(0, len(TypeList)):
            if TypeAllowed[tndx]:
                alist.append(TypeList[tndx])
                
        if len(alist)==0:
            alist = [UNK]
        
        AList.append(alist)
        
    return AList


# MList[p] is a list of matched types for pigment p
def match_types(PigNdcs, AList, instruc):
    
    MList = []
    # Loop through pigments
    for p in range(0, len(PigNdcs)):
        ndx = PigNdcs[p]
        
        tstruc = instruc[':'+str(ndx+1)]
        
        # alist is a list of pigment types allowed for pigment p
        alist = AList[p]
        
        # mlist will store a list of types that match the pigment exactly
        mlist = []
        
        # Loop through possible types
        for t in range(0, len(alist)):
            
            # Start assuming the pigment matches the type
            match = True
            typ = alist[t]
            for name in typ.xnames:
                
                # If any atom is not found, set match to False and break
                if len(tstruc['@'+name].atoms)==0:
                    match = False
                    break
                
            # If we have a match, append it to the pigment's match list
            if match:
                mlist.append(typ)
                
        MList.append(mlist)
    return MList

# 1. Assign all pigments to the top entry in MList[p] if non-empty
# 2. For each pigment still unassigned, check whether any pigments with the 
#    same name have already been assigned. If so, assign to the same type. 
# 3. For each pigment still unassigned, assign to the top entry in AList[p] if non-empty
def assign_pigments(PigNdcs, MList, AList, instruc):
    TList = []
    
    # 1. Assign by MList
    for p in range(0, len(PigNdcs)):
        if len(MList[p])>0:
            TList.append(MList[p][0])
        else:
            TList.append([])
            
    # 2. Assign by name
    for p in range(0, len(PigNdcs)):
        if TList[p]==[]:
            rname = struc.residues[PigNdcs[p]].name
            
            # Check if any assigned pigment has a matching name 
            for pref in range(0, len(PigNdcs)):
                if (TList[pref]!=[]) and (rname==struc.residues[PigNdcs[pref]].name):
                    TList[p] = TList[pref]
                    break
                    
    # 3. If pigments are still unmatched, assign to top alist entry
    for p in range(0, len(PigNdcs)):
        if TList[p]==[]:
            TList[p] = AList[p][0]
                
    return TList

In [4]:
# These names are considered essential to all Chl/BChl molecules
# Without these, we can do no calculations. 
ChlBaseNames = []
for ring in ['A', 'B', 'C', 'D']:
    for at in ['N', 'C1', 'C2', 'C3', 'C4', 'CH']:
        ChlBaseNames.append(at+ring)
ChlBaseNames.append('CAD')
ChlBaseNames.append('CBD')

# These are the atoms of the phytol tail
PhytolNames = []
for n in range(1, 21):
    PhytolNames.append('C'+str(n))
    
# All pigments in the class have these atoms, but we can identify the class without them
ChlOptNames = ['CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'CMA', 'CMB', 'CAB', 'CMC', 'CAC', 'CBC', 'CMD', 'OBD'] + PhytolNames


# The tetpy class describes common tetrapyrrole pigments
tetpy = pigment_class('tetrapyrrole', ChlBaseNames, ChlOptNames)


# These atoms can be used to distinguish between different pigment types
# Note: 
#    The heavy atoms of Chl a, Chl c1, Chl c2, and BChl g are identical (excluding the phytol tail)
#    The heavy atoms of BChl a and BChl b are identical
CLANames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED']
CLBNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED', 'OMC']
CLDNames = ['MG',        'CGD', 'O1D', 'O2D', 'CED',        'OBB']
CLFNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED',               'OMB'] 
BCANames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED',        'OBB']
BCBNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED',        'OBB']
BCCNames = ['MG', 'CBB',                                    'OBB',        'CIB']
BCDNames = ['MG', 'CBB',                                    'OBB',               'CND']
BCENames = ['MG', 'CBB',                             'OMC', 'OBB',        'CIB', 'CND']
BCFNames = ['MG', 'CBB',                             'OMC', 'OBB',               'CND']
BCGNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED']

# The corresponding Pheo names are the same, excluding the first entry (MG)
PHANames = CLANames[1:]
PHBNames = CLBNames[1:]
BPANames = BCANames[1:]

# Madjet et al. J. Phys. Chem. B, 110, 34, 17268–17281 [https://doi.org/10.1021/jp0615398]
CLA = pigment_type(
    'Chl a',
    'CLA',
    tetpy,
    CLANames,
    [],
    ['MG',    'CHA',    'CHB',    'CHC',    'CHD',  'NA',   'C1A',     'C2A',   'C3A',  'C4A',  'CMA',    'CAA',   'CBA',   'CGA',   'O1A',   'O2A',  'NB',     'C1B',  'C2B',  'C3B',   'C4B',   'CMB',    'CAB',   'CBB',  'NC',    'C1C',    'C2C',   'C3C',   'C4C',    'CMC',    'CAC',  'CBC',   'ND',     'C1D',    'C2D',   'C3D',    'C4D',     'CMD',   'CAD',     'OBD',    'CBD',    'CGD',  'O1D',    'O2D',  'CED',   'C1'],
    np.array([-21.674, 106.779,  -48.696,  -98.725,  72.726, 31.683,  -130.820, 10.048,  2.389,  77.983,  5.556,   -1.050,  0.735,   -8.040,  -1.330,  7.091,  -62.297,  81.122, 4.777,  -9.204,  106.271,  16.963,  10.681,  34.947, -12.166,  83.646,  -7.420,  -1.126,   -44.008,  -5.161,  8.020,  1.012,   108.292,  -110.812, -11.981,  8.799,   -125.044,  -25.156,  -19.250,  -20.044,  -11.238,  5.979,   -5.388,  1.811,  -5.256,  -1.424])*(1e-3),
    np.array([0.796,   -0.366,   0.046,    0.354,    0.259,  -0.245,  0.106,    0.186,   0.245,  -0.207,  -0.023,  -0.036,  -0.080,  0.673,   -0.522,  -0.374,  -0.281,  -0.038, 0.178,  -0.519,  0.096,    0.098,   0.110,   0.032,  -0.214,  -0.457,   0.246,   -0.318,   0.098,    0.061,   0.114,  0.007,  -0.348,    -0.390,    0.318,   -0.500,  0.420,     0.069,    0.543,    -0.460,   0.133,    0.424,   -0.428,  -0.343,  0.279,  0.258]),
    np.array([0.789,   -0.359,   0.055,    0.292,    0.179,  -0.256,  0.136,    0.188,   0.244,  -0.195,  -0.022,  -0.033,   -0.078,  0.671,   -0.521,  -0.373,  -0.277,  -0.041, 0.139,  -0.541,  0.132,   0.098,    0.117,   0.018,  -0.260,  -0.391,   0.250,   -0.331,   0.198,    0.064,   0.114,  0.008,   -0.353,   -0.333,    0.287,   -0.504,  0.449,     0.068,    0.517,    -0.468,   0.132,    0.427,   -0.429,  -0.344,  0.279,  0.25]),
    4.3e-18 # statC*cm
)

# Renger t al. J. Phys. Chem. B 2007, 111, 35, 10487–10501 [https://doi.org/10.1021/jp0717241] 
CLB = pigment_type(
    'Chl b',
    'CLB',
    tetpy,
    CLBNames, 
    [],
    ['MG', 'CHA', 'CHB', 'CHC', 'CHD', 'NA', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'NB', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'CBB', 'NC', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'OMC', 'CAC', 'CBC', 'ND', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'OBD', 'CBD', 'CGD', 'O1D', 'O2D', 'CED'],
    (1e-3)*np.array([-15.512,75.628,-57.844,-107.433,86.081,11.573,-99.670,7.454,7.749,77.389,3.351,-2.720,-5.754,4.859,0.650,-3.916,-73.990,85.402,-2.140,-5.618,108.571,14.838,1.978,24.357,-21.650,79.538,5.163,-5.803,-31.957,-13.413,-1.389,6.975,2.317,90.881,-88.170,-13.166,15.953,-98.277,-16.435,-28.234,-14.273,-0.035,7.743,-7.166,0.641,-4.526]),
    np.array([]),
    np.array([]),
    3.60e-18
)

CLD = pigment_type(
    'Chl d',
    'CLD',
    tetpy,
    CLDNames, 
    [],
    [],
    np.array([]),
    np.array([]),
    np.array([]),
    0.0
)

CLF = pigment_type(
    'Chl f',
    'CLF',
    tetpy,
    CLFNames,
    [],
    [],
    np.array([]),
    np.array([]),
    np.array([]),
    0.0
)


# Madjet et al. J. Phys. Chem. B, 110, 34, 17268–17281 [https://doi.org/10.1021/jp0615398] 
BCA = pigment_type(
    'BChl a',
    'BCA',
    tetpy,
    BCANames, 
    [],
    ['MG', 'CHA', 'CHB', 'CHC', 'CHD', 'NA', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'NB', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'OBB', 'CBB', 'NC', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'CAC', 'CBC', 'ND', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'OBD', 'CBD', 'CGD', 'O1D', 'O2D', 'CED', 'C1'],
    (1e-3)*np.array([-42.217,141.478,-14.988,-45.427,41.071,65.489,-134.776,19.419,2.723,17.043,11.656,-15.907,9.451,-5.063,5.421,-3.656,-28.216,87.663,-0.905,67.447,33.680,32.745,-25.091,37.208,19.037,16.855,80.984,-19.196,28.047,-116.550,7.787,0.863,-9.640,135.026,-111.514,-22.400,36.878,-200.720,-31.393,-35.896,-20.922,-2.346,-2.023,-8.885,7.086,-7.868,0.543]),
    np.array([0.700,-0.286,0.316,0.310,0.287,-0.411,0.393,0.006,0.191,-0.224,-0.017,-0.009,-0.023,0.624,-0.476,-0.432,0.139,-0.486,0.120,0.089,-0.463,0.068,0.297,-0.400,0.034,-0.163,-0.187,0.139,0.123,-0.227,0.001,0.038,-0.017,-0.097,-0.406,0.228,-0.301,0.085,0.074,0.417,-0.433,0.190,0.355,-0.399,-0.328,0.285,0.276]),
    np.array([0.683,-0.313,0.295,0.257,0.257,-0.439,0.413,0.015,0.194,-0.193,-0.013,-0.005,-0.024,0.627,-0.477,-0.433,0.125,-0.463,0.093,0.062,-0.393,0.067,0.274,-0.413,0.034,-0.202,-0.146,0.144,0.126,-0.167,0.004,0.042,-0.016,-0.109,-0.353,0.210,-0.297,0.132,0.076,0.390,-0.442,0.192,0.357,-0.401,-0.331,0.285,0.276]),
    5.477e-18
)

BCB = pigment_type(
    'BChl b',
    'BCB',
    tetpy,
    BCBNames, 
    [],
    [],
    np.array([]),
    np.array([]),
    np.array([]),
    0.0
)

BCC = pigment_type(
    'BChl c',
    'BCC',
    tetpy,
    BCCNames,
    [],
    [],
    np.array([]),
    np.array([]),
    np.array([]),
    0.0
)

BCD = pigment_type(
    'BChl d',
    'BCD',
    tetpy,
    BCDNames, 
    [],
    [],
    np.array([]),
    np.array([]),
    np.array([]),
    0.0
)

BCE = pigment_type(
    'BChl e',
    'BCE',
    tetpy,
    BCENames, 
    [],
    [],
    np.array([]),
    np.array([]),
    np.array([]),
    0.0
)

BCF = pigment_type(
    'BChl f',
    'BCF',
    tetpy,
    BCFNames, 
    [],
    [],
    np.array([]),
    np.array([]),
    np.array([]),
    0.0
)

BCG = pigment_type(
    'BChl g',
    'BCG',
    tetpy,
    BCGNames, 
    [],
    [],
    np.array([]),
    np.array([]),
    np.array([]),
    0.0
)

# Renger, T. Biophysical Journal, 95, 105-119, [https://doi.org/10.1529/biophysj.107.123935]
PHA = pigment_type(
    'Pheo a',
    'PHA',
    tetpy,
    PHANames,
    [],
    ['CHA', 'CHB', 'CHC', 'CHD', 'NA', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'NB', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'CBB', 'NC', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'CAC', 'CBC', 'ND', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'OBD', 'CBD', 'CGD', 'O1D', 'O2D', 'CED'],
    (1e-3)*np.array([76.834,-38.800,-89.458,62.844,47.290,-86.246,7.560,-1.965,70.215,4.115,-2.753,1.651,-3.830,-1.585,2.947,-62.801,66.190,8.864,-9.269,90.283,15.078,9.556,28.399,-25.439,73.639,9.381,-20.445,-24.827,-3.833,6.785,-0.061,87.071,-87.246,-8.812,2.558,-94.919,-9.925,-16.035,-18.630,-8.844,4.351,-4.024,2.224,-5.226]),
    np.array([]),
    np.array([]),
    3.50e-18
)

UNK = pigment_type(
    'Unknown',
    'UNK',
    tetpy,
    [],
    [],
    [],
    np.array([]),
    np.array([]),
    np.array([]),
    0.0
)


TetPyList = [
    CLA, CLB, #CLD, CLF, 
    BCA, #BCB, BCC, BCD, BCE, BCF, BCG,
    PHA, 
]

In [5]:
# Global variables:

# Main ParmEd structure
struc = pmd.structure.Structure()

# Representation list for main structure
mainreps = list()

# Representation list for dipoles
dipreps = list()

# List of chains in struc
ChainList = []

# List of identified pigments
PigList = []

##################################################################
# Main frame layout:
##################################################################

# mainbox is the top widget. It contains:
#  pdbview -- an NGLWidget used to display the loaded structures
#  mainacc -- an accordion widget containing the following VBoxes:
#    strucbox -- contains the "Load" interface
#    selbox -- contains the "Select" interface
#    writebox -- contains the "Write PDB" interface
#    excbox -- contains the "Prepare Exciton Model" interface
#    mdbox -- contains the "Prepare MD Model" interface

# Structure viewer:
pdbview = nv.NGLWidget()
pdbview._set_size('500px', '500px')


##################################################################
# strucbox: Interface for loading molecular structures
##################################################################

# pdbid: Text entry box for PDB ID 
# pdbidlbl: Label for pdbid
# pdbfetch: Button to fetch PDB from the RCSB databank
# pdbup: Button to upload PDB file
# pdbuplbl: Label for pdbup


pdbid = widgets.Text(
    value='2DRE',
    placeholder='',
    layout = widgets.Layout(width='1.5cm'),
    disabled=False
)

pdbidlbl = widgets.Label(value='Enter PDB ID:', layout=Layout(width='2.5cm'))

pdbfetch = widgets.Button(
    description='Fetch',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to display the pdb file',
    layout = widgets.Layout(width='2.25cm'),
    icon='' # (FontAwesome names without the `fa-` prefix)
)

pdbup = widgets.FileUpload(
    accept='.pdb, .gro',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

pdbuplbl = widgets.Label(value='Or upload file:', layout=Layout(width='2.5cm'))

# Executed with pdbfetch is clicked
def pdbfetch_onclick(b):

    url = 'http://files.rcsb.org/download/' + pdbid.value + '.pdb'
    r = requests.get(url, allow_redirects=True)
    if(r.status_code!=200):
        print('Invalid PDB code. Please try again.')
    else:
        fname = 'pdb/'+pdbid.value+'.pdb'
        wfd = open(fname, 'wb')
        wfd.write(r.content)
        wfd.close()
        init_struc(fname)

# Executed when pdbup is clicked
def pdbup_on_value_change(change):
    for item in pdbup.value:
        fname = item
    
    with open("pdb/"+fname, "wb") as fp:
        fp.write(pdbup.value[fname]["content"])
    fp.close()
    init_struc("pdb/"+fname)
    
pdbup.observe(pdbup_on_value_change, 'value')


grodrop = widgets.Dropdown(
    options=[],
    value = None,
    description='Or pick MD run:',
    disabled=False,
    layout=Layout(width='7cm')
)
grodrop.style.description_width='2.5cm'

def refresh_grolist(b):
    flist = !{"ls md/*/pdb2gmx/em.gro"}
    grolist = []
    for file in flist:
        prefix = file.split('/')[1]
        grolist.append(prefix)
    grodrop.options = grolist
    
# Executed when groload is clicked
def groload_onclick(b):
    prefix = grodrop.value
    if prefix!=None and len(prefix)>0:
        fname = 'md/'+prefix+'/pdb2gmx/em.gro'
        init_struc(fname)
        add_charges('md/'+prefix+'/pdb2gmx/charges.txt')
        x, d, f = calculate_tresp_shift()
        print()
        print('***************************************************')
        print('TrESP shift analysis for MD run ' + prefix)
        print('Calculated frequencies (cm$^{-1}$):')
        for val in f:
            print("\t", val)
        print()
        print('Average: \t', np.mean(f), 'cm$^{-1}$')
        print()
        print('All frequency values are relative to vacuum.')
        print()
    
def add_charges(fname):
    global struc
    charges = np.loadtxt(fname)
    if len(charges)==len(struc.atoms):
        for n in range(0, len(struc.atoms)):
            atom = struc.atoms[n]
            atom.charge = charges[n]
    
gro_refresh = widgets.Button(description='Refresh List')
gro_refresh.on_click(refresh_grolist)
refresh_grolist(0)

gro_load = widgets.Button(description='Load')
gro_load.on_click(groload_onclick)

strucbox = widgets.VBox([
    widgets.HBox([pdbidlbl, pdbid, pdbfetch]), 
    widgets.HBox([pdbuplbl, pdbup]),
    #grodrop,
    #widgets.HBox([gro_load, gro_refresh]),
])




def build_dipstruc(struc):
    
    dipstruc = pmd.structure.Structure()
    
    dipcoords = []
    porphlist = list()
    porphtxt = '('
    PorphAts = ['NA', 'NB', 'NC', 'ND']
    for res in struc.residues:

        # Check if it's a porphyrrin
        foundNs = np.zeros((len(PorphAts),))
        patnums = np.zeros((len(PorphAts),), dtype='int')
        for at in res:
            for n in range(0, len(PorphAts)):
                if at.name==PorphAts[n]:
                    foundNs[n] = 1
                    patnums[n] = at.idx

        # If we located all four ring N atoms
        if np.sum(foundNs)==4:
            cenvec = 0.5*(struc.coordinates[patnums[1]] + struc.coordinates[patnums[3]])
            dipvec = (struc.coordinates[patnums[3]] - struc.coordinates[patnums[1]])
            vStart = cenvec - 1.25*dipvec
            vStop = cenvec + 1.25*dipvec
            Nats = 10
            for a in range(0, Nats+1):
                xyz = (float(a)/float(Nats))*vStart + (1 - float(a)/float(Nats))*vStop
                dipstruc.add_atom(pmd.topologyobjects.Atom(name='N'), 'Dip', res.idx, chain=res.chain)
                dipcoords.append(xyz)

            porphlist.append(res.idx)
            if len(porphtxt)>1:
                porphtxt += ' OR '
            porphtxt += str(res.idx+1)
    porphtxt += ')'

    # If any porphyrrins have been located, add the coordinates to the dipole list. 
    if len(dipcoords)>0:
        dipstruc.coordinates = np.array(dipcoords)
    else: 
        porphtxt = ""
        
    return dipstruc, porphtxt
    

# init_struc() loads a structure from the provided file name
# and initializes the structure view representations and chainlists
#
# Outside references:
#   sets pigbox.children and chainbox.children each time a new structure is loaded
#
# Relies on environment variables:
#   UNK
#   tetpy
#   TetPyList
def init_struc(fname):

    global mainreps
    global dipreps
    global struc
    global ChainList
    global chainbox
    global writebt
    global pigbox
    global PigList
    
    # Clear pdbview stage
    while len(pdbview._ngl_component_ids)>0:
        pdbview.remove_component(pdbview._ngl_component_ids[0])
    
    # Reset rep and chain lists
    mainreps = list()
    dipreps = list()
    
    # Reset MD run button
    mdrun_button.value = HTMLDeadPrompt.format(text='Simulate')
    
    struc = pmd.load_file(fname)
    dipstruc, porphtxt = build_dipstruc(struc)
    
    ChainList = []
    for res in struc.residues:
        # If the chain is not already listed, add it
        if ChainList.count(res.chain)==0:
            ChainList.append(res.chain)
    
    chaintxt = '('
    for chain in ChainList:
        if len(chaintxt)>1:
            chaintxt += ' OR '
        chaintxt += ':' + chain
    chaintxt += ')'

    pdbview.add_trajectory(struc)
    mainreps = list()
    mainreps.append({"type": "cartoon", "params": {"color": "grey", "sele": "(protein) AND " + chaintxt, "opacity": "0.2"}})
    if len(porphtxt)>0:
        mainreps.append({"type": "licorice", "params": {"color": "green", "sele": porphtxt + ' AND ' + chaintxt, "opacity": "1.0"}})
    pdbview.set_representations(mainreps, component=0)

    if len(dipstruc.atoms)>0:
        pdbview.add_trajectory(dipstruc)
        dipreps = [{"type": "licorice", "params": {"color": "red", "sele": chaintxt, "opacity": "1", "radius": "0.35"}}]
        pdbview.set_representations(dipreps, component=1)

    # Assign tetrapyrrole types:
    # 1. Identify tetrapyrrole rings
    PigNdcs = find_pigments(tetpy.BaseNames, struc)

    # 2. Check which types are definitely excluded for each pigment
    alist = eliminate_types(PigNdcs, tetpy, TetPyList, UNK, struc)

    # 3. Now check if all xatoms of each type are present
    mlist = match_types(PigNdcs, alist, struc)
    
    # 4. Based on this data, assign pigment types
    tlist = assign_pigments(PigNdcs, mlist, alist, struc)
    
    PigList = []
    # Record pigments:
    for p in range(0, len(PigNdcs)):
        ndx = PigNdcs[p]
        res = struc.residues[ndx]
        atom_ndcs = []
        atom_names = []
        for at in res:
            atom_ndcs.append(at.idx)
            atom_names.append(at.name)
        atom_ndcs = np.array(atom_ndcs, dtype='int')
        atcoords = struc.coordinates[atom_ndcs]
        PigList.append(pigment(ndx, tlist[p], alist[p], res.name, res.chain, res.number, atom_names, atcoords, res))
    
    build_pigbox()
    
    chaincbs = []
    for chain in ChainList:
        chaincbs.append(widgets.Checkbox(value=True, description=chain,indent=False, layout=Layout(width='100px')))
    for cb in chaincbs:
        cb.observe(update_chains)
    chainbox.children = chaincbs
    writebt.disabled = False
    
    excgo.disabled = False
    coupselect.disabled = False
    siteselect.disabled = False
    excfile.value = fname.split('.')[-2].split('/')[-1]
    writetxt.value = fname.split('.')[-2].split('/')[-1]+'.pdb'
    mdfile.value = fname.split('.')[-2].split('/')[-1]
    
    mdgobt.disabled = False
    waterselect.disabled = False
    ffselect.disabled = False
    

# Syncs structure display to selected chains in chainbox
def update_chains(b):
    global dipreps
    global mainreps 
    global ChainList
    
    ChainList = []
    for cb in chainbox.children:
        if cb.value==True:
            ChainList.append(cb.description)
        
    chaintxt = ''
    for chain in ChainList:
        if len(chaintxt)>0:
            chaintxt += " OR "
        chaintxt += ":" + chain
        
    # If no chain is selected, set to a nonsense chain
    # so that none will be displayed.
    if len(chaintxt)==0:
        chaintxt = ':XXXXXXXXXX'
        
    for rep in mainreps:
        #rep['params']['sele'] = re.sub('(:[^)]+)', chaintxt, rep['params']['sele'])
        splt = rep['params']['sele'].split('(')
        newstr = ''
        for n in range(1, len(splt)-1):
            newstr += "(" + splt[n]
        newstr += '(' + chaintxt + ")"
        rep['params']['sele'] = newstr
    pdbview.set_representations(mainreps, component=0)
    
    #dipreps[0]['params']['sele'] = re.sub('(:[^)]+)', chaintxt, rep['params']['sele'])
    if len(dipreps)>0:
        splt = dipreps[0]['params']['sele'].split('(')
        newstr = ''
        for n in range(1, len(splt)-1):
            newstr += "(" + splt[n]
        newstr += '(' + chaintxt + ")"
        dipreps[0]['params']['sele'] = newstr
        pdbview.set_representations(dipreps, component=1)
    
pdbfetch.on_click(pdbfetch_onclick)

##################################################################
# selbox: Interface for selecting chains and residues
##################################################################

# Label for chain list
chainlbl = widgets.Label(value='Chain:')

# The children (i.e., options) of chainbox are set in pdbfetch_onclick()
# when a new structure is loaded. 
chainbox = widgets.VBox([])

selall = widgets.Button(
    description='All',
    disabled=False,
    tooltip='Click to select all chains',
    layout = widgets.Layout(width='1.5cm'),
)

selnone = widgets.Button(
    description='None',
    disabled=False,
    tooltip='Click to select all chains',
    layout = widgets.Layout(width='1.5cm'),
)

def selall_onclick(b):
    for cb in chainbox.children:
        cb.value = True
selall.on_click(selall_onclick)

def selnone_onclick(b):
    for cb in chainbox.children:
        cb.value = False
selnone.on_click(selnone_onclick)

selbox = widgets.VBox([widgets.HBox([selall, selnone]), chainlbl, chainbox])


##################################################################
# writebox: Interface for writing PDB output
##################################################################

writetxt = widgets.Text(value='test.pdb', description='File Name:', disabled=False, layout=widgets.Layout(width='5cm'))
writebt = widgets.Button(description='Write',tooltip='Click to write PDB file',layout=widgets.Layout(width='1.5cm'), disabled=True)

def pdbwrite_onclick(b):
    fname = "pdb/" + writetxt.value
    if fname[-4:]!='.pdb':
        fname += '.pdb'
    
    # First identify which chains should be written
    # Loop through chain-selection check-boxes and
    # add a parmed structure object for each chain
    strucList = []
    for cb in chainbox.children:
        if cb.value==True:
            strucList.append(struc[cb.description,:,:])
    
    # Now combine all sub-structures into a single structure
    # for writing to pdb. 
    selstruc = []
    if len(strucList)>0:
        selstruc = strucList[0]
        for n in range(1, len(strucList)):
            selstruc = selstruc + strucList[n]
        selstruc.write_pdb(fname)
#         disptext = 'Structure successfully written to file <a href=\"' + fname + '\" target="_blank">'+fname+'</a>'
#         display_markdown(disptext, raw=True)
        
writebt.on_click(pdbwrite_onclick)

pdb_down = widgets.HTML(HTMLButtonPrompt.format(link='pdb/', text='Download'))

writebox = widgets.Box([widgets.HBox([writetxt, writebt]), pdb_down], 
                      layout=Layout(flex_flow='column',
                                   align_items='center'))


##################################################################
# excbox: Interface for building exciton models
##################################################################

def calculate_tresp():
    
    global PigList
    global ChainList
    
    # First create list of chain-selected pigments
    SelPigs = []
    for pig in PigList:
        if ChainList.count(pig.chain)>0:
            SelPigs.append(pig)
    Npigs = len(SelPigs)

    Npigs = len(SelPigs)
    CoupMat = np.zeros((Npigs,Npigs))
    Dips = []
    h = 6.62607015e-34 #J*s
    c = 2.998e10 #cm/s
    eo = 4.80320451e-10 # esu
    
    Erg2J = 1.0e-7
    ang2cm = 1.0e-8
    
    # Now check whether TrESP atoms are available for *all*
    # selected pigments. 
    missing_atoms = False
    
    # Loop over selected atoms
    for p in range(0, Npigs):
        pig = SelPigs[p]
        # Loop over TrESP atoms
        for name in pig.ptype.tnames:
            if pig.atnames.count(name)==0:
                print('Error: Could not locate TrESP atom ' + name + " in pigment " + pig.resname + " " + pig.chain + " " + str(pig.resnum))
                print('Aborting TrESP calculation. Please try point-dipole approximation (PDA) or add missing atoms.')
                missing_atoms = True
                break
            if missing_atoms:
                break
                
    if missing_atoms==False:
        # Build list of TrESP coordinates for each pigment
        # NB: Units are converted to cm!
        CoordList = []
        for p in range(0, Npigs):
            pig = SelPigs[p]
            coords = []
            for name in pig.ptype.tnames:
                ndx = pig.atnames.index(name)
                coords.append(pig.atcoords[ndx])
            CoordList.append(np.array(coords)*ang2cm)
            
        # Calculate pigment dipole lengths (as per default TrESP charges)
        # We'll divide charges for each pigment by the ratio of the *calculated*
        # dipole length to the *standard* dipole length. 
        # Charges are in units of cm^3/2 g^1/2 s^−1.
        DipLengths = []
        for p in range(0, Npigs):
            pig = SelPigs[p]
            dip = np.zeros((3,))
            for n in range(0, len(pig.ptype.tnames)):
                dip += eo*pig.ptype.q10[n]*CoordList[p][n]
                
            DipLengths.append(np.linalg.norm(dip))
            Dips.append(dip/np.linalg.norm(dip))
        Dips = np.array(Dips)
            
        # Calculate interactions
        for p1 in range(0, Npigs):
            pig1 = SelPigs[p1]
            df1 = pig1.ptype.diplength/DipLengths[p1]
            for p2 in range(0, p1):
                pig2 = SelPigs[p2]
                df2 = pig2.ptype.diplength/DipLengths[p2]
                for atm in range(0, len(pig1.ptype.q10)):
                    for atn in range(0, len(pig2.ptype.q10)):
                        Rmn = CoordList[p1][atm,:] - CoordList[p2][atn,:]
                        rmn = np.linalg.norm(Rmn)
                        CoupMat[p1,p2] += eo*eo*df1*df2*(pig1.ptype.q10[atm]*pig2.ptype.q10[atn]/rmn)*(Erg2J/h)/c
        
        CoupMat = CoupMat + np.transpose(CoupMat)
        
    return CoupMat, Dips

def build_excitons(b):
    
    prefix = excfile.value
    for chars in forbidden_strings:
        if len(prefix)==0 or prefix.find(chars)!=-1:
            excoutlbl.value = 'Please enter a valid file prefix for output. Avoid spaces and the special characters "~", "..", "/", and "\\".'
            return
        
    if os.path.isdir('exc/'+prefix+".exc")==True:
        if excovercb.value==False:
            excoutlbl.value = "Directory exists. Please check 'Overwrite Existing' or pick another directory name."
            return
        else:
            !{"rm -r exc/"+prefix+".exc"}
            
    !{"mkdir exc/"+prefix+".exc"}
        
    if os.path.isdir('exc/'+prefix+".exc")==False:
        excoutlbl.value = 'Error creating directory. Please choose a different export name.'
        return
    
    if coupselect.value=='TrESP':
        Coups, Dips = calculate_tresp()
    
    if siteselect.value=='Uniform':
        Freqs = []
        for p in range(0, len(PigList)):
            if ChainList.count(PigList[p].chain)>0:
                pname = PigList[p].ptype.stdname
                if pname=='CLA' or pname=='PHA':
                    freq = (1.0e+7)/670.0
                elif pname=='CLB' or pname=='PHB':
                    freqs = (1.0e+7)/655.0
                elif pname[0]=='B':
                    freq = (1.0e+7)/800.0
                else:
                    freq = (1.0e+7)/670.0
                Freqs.append(freq)
                
    # If we successfully calculated coupling constants
    if len(Dips)>0:
        comment_string = ''
        for p in range (0, len(PigList)):
            pig = PigList[p]
            if ChainList.count(pig.chain)>0:
                comment_string += pig.resname + ":" + pig.chain + str(pig.resnum) + ' '
        
        np.savetxt('exc/' + prefix + '.exc/coups.txt', Coups, header = comment_string, fmt='%10.3f')
        np.savetxt('exc/' + prefix + '.exc/freqs.txt', Freqs, header = comment_string, fmt='%10.3f')
        np.savetxt('exc/' + prefix + '.exc/dips.txt', Dips, header = comment_string, fmt='%10.3f')
        with open('exc/' + prefix + '.exc/names.txt', "w") as file:
            file.write(comment_string)
        excoutlbl.value = 'Exciton model stored in directory exc/'+prefix+'.exc/'
        
        
couplbl = widgets.Label(value='Coupling Model')
coupselect = widgets.RadioButtons(
    options=['TrESP'],
    value='TrESP',
    disabled=True,
    layout=Layout(width='2cm')
)

sitelbl = widgets.Label(value='Site Energy Model')
siteselect = widgets.RadioButtons(
    options=['Uniform'],
    value='Uniform',
    disabled=True,
    layout=Layout(width='2cm')
)

excfilelbl = widgets.Label(value='File prefix: ')

excfile = widgets.Text(
    value='test',
    placeholder='file prefix',
    layout = widgets.Layout(width='2.5cm'),
    disabled=False
)

excovercb = widgets.Checkbox(value=False, description='Overwrite Existing', layout=Layout(width='4cm'))
excovercb.style.description_width='0cm'

excgo = widgets.Button(
    description='Go!',
    disabled=True,
    tooltip='Click to build an exciton model',
    layout=Layout(width='150px')
)
#excgo.style.button_color='#CCCCCC'

excgo.on_click(build_excitons)

exc_down = widgets.HTML(HTMLButtonPrompt.format(link='./exc/', text='Download', layout=Layout(width='0.5cm')))

spec_button = widgets.HTML(HTMLButtonPrompt.format(link='Spectrum.ipynb', text='Simulate'))

excoutlbl = widgets.HTML(value='')

excbox = widgets.VBox(
    [
#         widgets.HBox([
#             widgets.VBox([couplbl, coupselect], layout=Layout(width='3cm', margin='5pt')),
#             widgets.VBox([sitelbl, siteselect], layout=Layout(width='3cm', margin='5pt')),
#         ], layout=Layout(width='7cm', margin='5pt'), ),
        widgets.HBox([excfilelbl, excfile]),
        excovercb,
        excgo,
        widgets.Label(layout=Layout(height='20px')),
        widgets.HBox([exc_down,spec_button]), 
        excoutlbl,
    ], layout=Layout(align_items='center', width='7.5cm'))


##################################################################
# mdbox: Interface for preparing MD models
##################################################################

fflbl = widgets.Label(value='Force Field')
ffselect = widgets.RadioButtons(
    options=['oplsaa'],
    value='oplsaa',
    disabled=True,
    layout=Layout(width='2.5cm')
)

waterlbl = widgets.Label(value='Water Model')
waterselect = widgets.RadioButtons(
    options=['spce'],
    value='spce',
    disabled=True,
    layout=Layout(width='2.5cm')
)

mdgobt = widgets.Button(
    description='Go!',
    disabled=True,
    tooltip='Click to build an MD model',
    layout=Layout(width='150px')
)

mdfilelbl = widgets.Label(value='File prefix: ')

mdfile = widgets.Text(
    value='test',
    placeholder='file prefix',
    layout = widgets.Layout(width='2.5cm'),
    disabled=False
)

mdovercb = widgets.Checkbox(value=False, description='Overwrite Existing', layout=Layout(width='4cm'))
mdovercb.style.description_width='0cm'

md_down = widgets.HTML(HTMLButtonPrompt.format(link='./md/', text='Download', layout=Layout(width='0.5cm')))

mdrun_button = widgets.HTML(HTMLDeadPrompt.format(text='Simulate'))

mdoutlbl = widgets.HTML(value='')

# Get rid of all but one altloc record
def strip_altlocs(struc):
    delMask = []
    for res in struc.residues:
        al = ''
        for atom in res:
            del_flag = False

            # If entry has an altloc, let's decide what to do with it
            if len(atom.altloc.strip())>0:

                # If altloc for this residue hasn't been set, then set it
                if len(al)==0:
                    al = atom.altloc.strip()

                # Otherwise, if it doesn't match, get rid of it
                elif al!=atom.altloc.strip():
                    del_flag = True
            delMask.append(del_flag)
    struc.strip(delMask)
    return struc
    
def mdgo_onclick(b):
    protfname = "protein.pdb"
    pigfname = "pigments.pdb"
    sysfname = "system.pdb"
    
    prefix = mdfile.value
    for chars in forbidden_strings:
        if len(prefix)==0 or prefix.find(chars)!=-1:
            mdoutlbl.value = 'Please enter a valid file prefix for output. Avoid spaces and the special characters "~", "..", "/", and "\\".'
            return
        
    if os.path.isdir('md/'+prefix)==True:
        if mdovercb.value==False:
            mdoutlbl.value = "Directory exists. Please check 'Overwrite Existing' or pick another directory name."
            return
        else:
            !{"rm -r md/"+prefix}
    !{"mkdir md/"+prefix}
    !{"mkdir md/"+prefix+"/pdb2gmx/"}
    !{"mkdir md/"+prefix+"/nvt/"}
    !{"mkdir md/"+prefix+"/npt/"}
    !{"cp misc/pymol/mutate.py md/"+prefix+"/pdb2gmx/"}
    !{"cp misc/gmx/ipynb/pdb2gmx.ipynb md/"+prefix+"/pdb2gmx/"}
    !{"cp misc/gmx/ipynb/add_hydrogens.ipynb md/"+prefix+"/pdb2gmx/"}
    !{"cp misc/gmx/ipynb/build_complex.ipynb md/"+prefix+"/pdb2gmx/"}
    !{"cp misc/gmx/ipynb/minimize_complex.ipynb md/"+prefix+"/pdb2gmx/"}
    !{"cp misc/gmx/ipynb/md_organizer.ipynb md/"+prefix+"/"}
    !{"cp misc/gmx/ipynb/parse_pqr.sh md/"+prefix+"/pdb2gmx/"}
    !{"cp misc/gmx/mdp/em.mdp md/"+prefix+"/pdb2gmx/"}
    !{"cp misc/gmx/mdp/nvt.mdp md/"+prefix+"/nvt/"}
    !{"cp misc/gmx/ipynb/nvt.ipynb md/"+prefix+"/nvt/"}
    !{"cp misc/gmx/mdp/npt.mdp md/"+prefix+"/npt/"}
    !{"cp misc/gmx/ipynb/npt.ipynb md/"+prefix+"/npt/"}
    
        
    if os.path.isdir('md/'+prefix)==False:
        mdoutlbl.value = 'Error creating directory. Please choose a different export name.'
        return
    
    # First identify which chains should be written
    # Loop through chain-selection check-boxes and
    # add a parmed structure object for each chain
    strucList = []
    for cb in chainbox.children:
        if cb.value==True:
            strucList.append(struc[cb.description,:,:])
    
    # Now combine all sub-structures into a single structure
    # and write to pdb. 
    selstruc = []
    if len(strucList)>0:
        selstruc = strucList[0]
        for n in range(1, len(strucList)):
            selstruc = selstruc + strucList[n]
            
        # First write total structure to file
        selstruc.write_pdb("md/"+prefix+"/pdb2gmx/"+sysfname)
        struc0 = pmd.load_file("md/"+prefix+"/pdb2gmx/"+sysfname)
        
        # Now re-load and strip altlocs (keep only one)
        newstruc = strip_altlocs(struc0)
        newstruc.write_pdb("md/"+prefix+"/pdb2gmx/"+protfname)
        
        # Make a list of all pigment residue names
        ResNames = []
        StdNames = []
        for pig in PigList:
            if ResNames.count(pig.resname)==0:
                ResNames.append(pig.resname)
                StdNames.append(pig.ptype.stdname)
        
        # Now reload and write separate files for recognized pigments
        # and for everything else
        if len(ResNames)>0:
            struc0 = pmd.load_file("md/"+prefix+"/pdb2gmx/"+protfname)
            strucList = []
            
            # First write PDB file for pigments
            for n in range(0, len(ResNames)):
                
                name = ResNames[n]
                stdname = StdNames[n]
                
                # Create a new structure of only this pigment type
                newstruc = struc0[':'+name]
                
                # Standardize pigment names
                for res in newstruc.residues:
                    res.name = stdname
                    
                strucList.append(newstruc)
                
            if len(strucList)>0:
                pigstruc = strucList[0]
                for n in range(1, len(strucList)):
                    pigstruc = pigstruc + strucList[n]
                    
                pigstruc.write_pdb("md/"+prefix+"/pdb2gmx/"+pigfname)
                for res in pigstruc.residues:
                    # If we don't have a topology for this pigmen on record, throw an error. 
                    if os.path.isfile('misc/gmx/itp/'+res.name+".itp")==False:
                        mdoutlbl.value = 'Could not locate topology reference file for pigment ' + res.name + ". Aborting."
                        return
                    else:
                        # If we haven't already added the top file for this pigment, add it
                        if os.path.isfile('md/'+prefix+"/pdb2gmx/"+res.name+".itp")==False:
                            out = !{"cp misc/gmx/itp/"+res.name+".itp md/"+prefix+"/pdb2gmx/"}
                
            # Now rewrite main file without pigments
            for n in range(0, len(ResNames)):
                name = ResNames[n]
                struc0.strip(':'+name)
            struc0.write_pdb("md/"+prefix+"/pdb2gmx/"+protfname)
                
        
    
    with open("md/"+prefix+"/pdb2gmx/ffparams.txt", 'w') as fd:
        fd.write('FF: ' + ffselect.value + '\n')
        fd.write('WATER: ' + waterselect.value + '\n')
    fd.close()
    mdoutlbl.value='MD inputs writen to directory ' + prefix + '.'
    mdrun_button.value = HTMLButtonPrompt.format(link='md/'+prefix+'/md_organizer.ipynb', text='Simulate')

mdgobt.on_click(mdgo_onclick)

mdbox = widgets.Box([widgets.HBox([
            widgets.VBox([fflbl, ffselect], layout=Layout(width='3cm', margin='5pt')),
            widgets.VBox([waterlbl, waterselect], layout=Layout(width='3cm', margin='5pt')),
        ], layout=Layout(width='6cm', margin='5pt'), ),
        widgets.HBox([mdfilelbl, mdfile]),
        mdovercb,
        mdgobt, 
        widgets.HBox([md_down, mdrun_button]),
        mdoutlbl], 
    layout=Layout(flex_flow='column',align_items='center'))



#######################################################
# Main Box
#######################################################


def update_pigtypes(change):
    global PigList
    
    for pig in PigList:
        if pig.widget.value!=pig.ptype.name:
            for typ in pig.alist:
                if typ.name==pig.widget.value:
                    pig.ptype = typ
                    break

def build_pigbox():
    global pigbox
    global PigList
    
    pigbox.layout = Layout(display='flex',
                    flex_flow='row wrap',
                    align_items='stretch',
                    width='100%')

    hlist = []
    for chain in ChainList:
        chwidglist = []
        for p in range(0, len(PigList)):
            pig = PigList[p]
            res = struc.residues[pig.idx]
            if res.chain==chain:
                dropbox = widgets.Dropdown(
                            options=[typ.name for typ in pig.alist],
                            value=pig.ptype.name,
                            description=res.name + ' ' + chain + str(res.number),
                            disabled=False,
                            layout=Layout(width='4.5cm')
                        )
                dropbox.observe(update_pigtypes, 'value')
                pig.widget = dropbox
                chwidglist.append(dropbox)
        if len(chwidglist)>0:
            lbl = widgets.Label(value='Chain ' + chain + ":")
            hlist.append(widgets.VBox([lbl] + chwidglist))
    pigbox.children = hlist

mainacc = widgets.Accordion(children=[strucbox, selbox, writebox, excbox], layout=Layout(width='8cm'))
mainacc.set_title(0, 'Load')
mainacc.set_title(1, 'Select')
mainacc.set_title(2, 'Write PDB')
mainacc.set_title(3, 'Build Exciton Model')
mainacc.set_title(4, 'Build MD Model')

def on_accordion_change(change):
    if change["new"]!=4:
        mdrun_button.value = HTMLDeadPrompt.format(text='Simulate')
    
mainacc.observe(on_accordion_change, names="selected_index")

pigbox = widgets.HBox()
pigacc = widgets.Accordion(children=[pigbox])
pigacc.set_title(0, 'Pigment List')

mainbox = widgets.VBox([widgets.HBox([pdbview, mainacc]), pigacc])
display(mainbox)

excfile.value = '2DRE'
writetxt.value = '2DRE.pdb'
mdfile.value = '2DRE'


In [6]:
def locate_ligand(pig):
    if pig.atnames.count('MG')!=0:
        MGndx = pig.atnames.index('MG')
        data_frame = struc.to_dataframe()
        bgndcs = (data_frame.resid != pig.res.idx)
        dvec = np.sqrt(np.sum(np.power(pig.atcoords[MGndx] - struc.coordinates[bgndcs],2),1))
        ndx0 = np.argmin(dvec)
        at0 = struc[bgndcs].atoms[ndx0]
        return at0.residue
    else:
        return []
        

def calculate_tresp_shift():
    
    global PigList
    global ChainList
    
    # First create list of chain-selected pigments
    SelPigs = []
    for pig in PigList:
        if ChainList.count(pig.chain)>0:
            SelPigs.append(pig)
    Npigs = len(SelPigs)

    Npigs = len(SelPigs)
    CoupMat = np.zeros((Npigs,Npigs))
    Dips = []
    h = 6.62607015e-34 #J*s
    c = 2.998e10 #cm/s
    eo = 4.80320451e-10 # esu
    
    Erg2J = 1.0e-7
    ang2cm = 1.0e-8
    
    # Now check whether TrESP atoms are available for *all*
    # selected pigments. 
    missing_atoms = False
    
    # Loop over selected atoms
    for p in range(0, Npigs):
        pig = SelPigs[p]
        # Loop over TrESP atoms
        for name in pig.ptype.tnames:
            if pig.atnames.count(name)==0:
                print('Error: Could not locate TrESP atom ' + name + " in pigment " + pig.resname + " " + pig.chain + " " + str(pig.resnum))
                print('Aborting TrESP calculation. Please try point-dipole approximation (PDA) or add missing atoms.')
                missing_atoms = True
                break
            if missing_atoms:
                break
    
    if missing_atoms==False:
        # Build list of TrESP coordinates for each pigment
        CoordList = []
        for p in range(0, Npigs):
            pig = SelPigs[p]
            coords = []
            for name in pig.ptype.tnames:
                ndx = pig.atnames.index(name)
                coords.append(pig.atcoords[ndx])
            CoordList.append(np.array(coords))
            
        # Calculate pigment dipole lengths (as per default TrESP charges)
        # We'll divide charges for each pigment by the ratio of the *calculated*
        # dipole length to the *standard* dipole length. 
        # Charges are in units of cm^3/2 g^1/2 s^−1.
        DipLengths = []
        for p in range(0, Npigs):
            pig = SelPigs[p]
            dip = np.zeros((3,))
            for n in range(0, len(pig.ptype.tnames)):
                dip += eo*pig.ptype.q10[n]*CoordList[p][n]*ang2cm
                
            DipLengths.append(np.linalg.norm(dip))
            Dips.append(dip/np.linalg.norm(dip))
        Dips = np.array(Dips)
            
        # Calculate interactions
        for p1 in range(0, Npigs):
            pig1 = SelPigs[p1]
            df1 = pig1.ptype.diplength/DipLengths[p1]
            for p2 in range(0, p1):
                pig2 = SelPigs[p2]
                df2 = pig2.ptype.diplength/DipLengths[p2]
                for atm in range(0, len(pig1.ptype.q10)):
                    for atn in range(0, len(pig2.ptype.q10)):
                        Rmn = CoordList[p1][atm,:] - CoordList[p2][atn,:]
                        rmn = np.linalg.norm(Rmn)*ang2cm
                        CoupMat[p1,p2] += eo*eo*df1*df2*(pig1.ptype.q10[atm]*pig2.ptype.q10[atn]/rmn)*(Erg2J/h)/c
                     
        CoupMat += np.transpose(CoupMat)
        
        eps_eff = 2.5
        tFreqs = np.zeros((Npigs,))
        for p in range(0, Npigs):
            
            pig = SelPigs[p]
            shift = 0.0
            
            lig = locate_ligand(pig)
            data_frame = struc.to_dataframe()
            
            # We exclude the pigment itself and its ligand
            #ndcs = np.logical_and(data_frame.resid != pig.res.idx, data_frame.resid != lig.idx)
            ndcs = data_frame.resid != pig.res.idx
            
            for atm in range(0, len(pig.ptype.q00)):
                dQ = pig.ptype.q11[atm] - pig.ptype.q00[atm]
                Rvec = (CoordList[p][atm,:] - struc.coordinates[ndcs,:])*ang2cm
                Dvec = np.sqrt(np.sum(np.power(Rvec,2),1))
                Qvec = data_frame[ndcs].charge
                shift += eo*eo*dQ*np.sum(np.divide(Qvec,Dvec))*Erg2J/(h*c*eps_eff)
            tFreqs[p] = shift
        
    return CoupMat, Dips, tFreqs
